In [0]:
from azure.storage.blob import BlobServiceClient, generate_blob_sas, BlobSasPermissions
from azure.identity import ClientSecretCredential
from datetime import timedelta, timezone, datetime as dt

class AzureHelper:

    def get_secret(self, key):
        return dbutils.secrets.get(scope="kv-databricks-learn", key=key)
    
    def __init__(self):
        self.storage_account = self.get_secret("storageaccount")
        self.azure_blob_container = self.get_secret("azureblobcontainer")
        self.application_id = self.get_secret("applicationid")
        self.directory_id = self.get_secret("directoryid")
        self.service_credential = self.get_secret("servicecredential")

        self.models_folder = "models"
        self.account_url = f"https://{self.storage_account}.blob.core.windows.net"

        self.iot_event_hub_con_str = self.get_secret("ioteventhubconstr")
        self.iot_hub_con_str = self.get_secret("iothubconstr")


        # --- AUTH (already configured in your message) ---

        spark.conf.set(f"fs.azure.account.auth.type.{self.storage_account}.dfs.core.windows.net", "OAuth")
        spark.conf.set(f"fs.azure.account.oauth.provider.type.{self.storage_account}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
        spark.conf.set(f"fs.azure.account.oauth2.client.id.{self.storage_account}.dfs.core.windows.net", self.application_id)
        spark.conf.set(f"fs.azure.account.oauth2.client.secret.{self.storage_account}.dfs.core.windows.net", self.service_credential)
        spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{self.storage_account}.dfs.core.windows.net", f"https://login.microsoftonline.com/{self.directory_id}/oauth2/token")

    def get_target_blob_file(self, blob_name):
        print('get_target_blob_file')
        target_blob_file = f"abfss://{self.azure_blob_container}@{self.storage_account}.dfs.core.windows.net/{blob_name}"
        return target_blob_file

    def get_blob_service_client(self):
        print('get_blob_service_client')

        print('getting credential...')
        credential = ClientSecretCredential(
            tenant_id=self.directory_id,
            client_id=self.application_id,
            client_secret=self.service_credential
        )

        print('creating blob service client...')
        blob_service_client = BlobServiceClient(
            account_url=self.account_url,
            credential=credential
        )

        return blob_service_client
    
    def get_blob_name(self, model_name, model_version):
        print('get_blob_name')
        blob_name = f"{self.models_folder}/{model_name}_{model_version}.zip"
        return blob_name
    
    def generate_sas_token(self, blob_name: str, hours: int = 1):
        print('generate_sas_token')

        blob_service_client = self.get_blob_service_client()

        # get a user delegation key
        udk = blob_service_client.get_user_delegation_key(
            key_start_time=dt.now(timezone.utc),
            key_expiry_time=dt.now(timezone.utc) + timedelta(hours=1)
        )

        # generate SAS
        print('generating sas token...')
        sas_token = generate_blob_sas(
            account_name=self.storage_account,
            container_name=self.azure_blob_container,
            blob_name=blob_name,
            user_delegation_key=udk,
            permission=BlobSasPermissions(read=True, list=True),
            expiry=dt.now(timezone.utc) + timedelta(hours=1)
        )

        sas_url = f"{self.account_url}/{self.azure_blob_container}/{blob_name}?{sas_token}"
        print("✅ SAS URL:", sas_url)

        return sas_url
    

# model_name = "aisdk_model_state_identifier-edge"
# model_version = "1"

# azure_helper = AzureHelper()

# blob_name = azure_helper.get_blob_name(model_name, model_version)

# azure_helper.generate_sas_token(blob_name)

In [0]:
import sys

print(sys.path)